In [42]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [35]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [20]:
old_len = len(df)

In [36]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [16]:
df.duration.mean()

In [37]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [38]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].fillna(-1).astype('int')

In [52]:
df[categorical] = df[categorical].astype('str')

In [55]:
train_dicts = df[categorical].to_dict(orient='records')

In [56]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts) 

In [57]:
X_train.shape

In [60]:
y_train = df.duration.values

In [59]:
len(dv.feature_names_)

In [61]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [62]:
y_pred = lr.predict(X_train)

In [63]:
mean_squared_error(y_train, y_pred, squared=False)

In [64]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [65]:
df_val = read_data('./data/fhv_tripdata_2021-02.parquet')

In [68]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [69]:
X_val = dv.transform(val_dicts) 

In [70]:
y_pred = lr.predict(X_val)

In [71]:
y_val = df_val.duration.values

In [72]:
mean_squared_error(y_val, y_pred, squared=False)